<a href="https://colab.research.google.com/github/shivkumarganesh/Deep-Learning-Course/blob/main/Assignment-6/Part_A_Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from scipy.special import softmax
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.datasets import mnist
import numpy as np


# Creating a Custom Tensor

In [30]:
# Reference from the Tutorial Provided
class CustomTensor (object):
    # Initialization of the CustomTensor (Constructor)
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True 
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
 
            if(grad is None):
                grad = CustomTensor(np.ones_like(self.data))

            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if (self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if (self.creation_op == "add"):
                  self.creators[0].backward(self.grad, self)
                  self.creators[1].backward(self.grad, self)
                    
                if (self.creation_op == "sub"):
                  self.creators[0].backward(CustomTensor(self.grad.data), self)
                  self.creators[1].backward(CustomTensor(self.grad.__neg__().data), self)

                if (self.creation_op == "mul"):
                  new = self.grad * self.creators[1]
                  self.creators[0].backward(new , self)
                  new = self.grad * self.creators[0]
                  self.creators[1].backward(new, self)                    
                    
                if (self.creation_op == "mm"):
                  c0 = self.creators[0]
                  c1 = self.creators[1]
                  new = self.grad.mm(c1.transpose())
                  c0.backward(new)
                  new = self.grad.transpose().mm(c0).transpose()
                  c1.backward(new)
                    
                if (self.creation_op == "transpose"):
                  self.creators[0].backward(self.grad.transpose())

                if ("sum" in self.creation_op):
                  dim = int(self.creation_op.split("_")[1])
                  self.creators[0].backward(self.grad.expand(dim,self.creators[0].data.shape[dim]))


                if ("expand" in self.creation_op):
                  dim = int(self.creation_op.split("_")[1])
                  self.creators[0].backward(self.grad.sum(dim))
                

                if self.creation_op == "softmax":
                  self.creators[0].backward(self.grad)
                    
                if(self.creation_op == "neg"):
                  self.creators[0].backward(self.grad.__neg__())
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return CustomTensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return CustomTensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return CustomTensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return CustomTensor(self.data * -1)
    
    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return CustomTensor(self.data - other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return CustomTensor(self.data - other.data)
    
    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return CustomTensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return CustomTensor(self.data * other.data)    

    def sum(self, dim):
        if(self.autograd):
            return CustomTensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return CustomTensor(self.data.sum(dim))
    
    def expand(self, dim,copies):

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)
        
        if(self.autograd):
            return CustomTensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return CustomTensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return CustomTensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        
        return CustomTensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return CustomTensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return CustomTensor(self.data.dot(x.data))
    
    def softmax(self):
        x = self.data - self.data.max(axis=1, keepdims=True)
        y = np.exp(x)
        v = y / y.sum(axis=1, keepdims=True)

        if self.autograd:
            return CustomTensor(v,
                          autograd=True,
                          creators=[self],
                          creation_op="softmax")
        return CustomTensor(v)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  

In [31]:
x = CustomTensor(['hello'])
x1 = CustomTensor([1,2,3,4,5], autograd=True)
x2 = CustomTensor([1.,2.,3.,4.,5.], autograd=True)
x3 = CustomTensor([5,4,3,2,1], autograd=True)

In [32]:
print(x)
print(x1)
print(x2)
print(x3)

['hello']
[1 2 3 4 5]
[1. 2. 3. 4. 5.]
[5 4 3 2 1]


# Importing MNIST Dataset

In [33]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28*28) / 255.
X_test = X_test.reshape(X_test.shape[0], 28*28) / 255.
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)
print('Y_train.shape', Y_train.shape)
print('Y_test.shape', Y_test.shape)

X_train.shape (60000, 784)
X_test.shape (10000, 784)
Y_train.shape (60000, 10)
Y_test.shape (10000, 10)


# MNIST Data classification

In [34]:
data = CustomTensor(X_train, autograd=True)
target = CustomTensor(Y_train, autograd=True)

w = list()
w.append(CustomTensor(np.random.rand(X_train.shape[1], 64), autograd=True))
w.append(CustomTensor(np.random.rand(64, 10), autograd=True))

lr = 1e-3

for i in range(100):
    # Predict
    pred = data.mm(w[0]).mm(w[1]).softmax()

    # Compare
    loss = ((pred - target) * (pred - target)).sum(0)

    # Learn
    loss.backward(CustomTensor(np.ones_like(loss.data)))

    for w_ in w:
        w_.data -= w_.grad.data * lr
        w_.grad.data *= 0

    print(np.mean(loss.data))

10445.45999677915
10651.6
10651.6
10651.6
10815.4
10829.8
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10829.8
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10829.8
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10829.8
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10829.8
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10829.8
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10815.4
10829.8
10815.4
10815.4
10815.4


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:175: RuntimeWarning: invalid value encountered in subtract


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
